# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
vacationDF = pd.read_csv("weather_df")
vacationDF.head()

,City,Latitude,Longitude,Temperature (F),Humidity (%),Wind Speed (mph),Cloudiness (%)
0,Loughrea,53.20,-8.57,54.37,100,6.93,75
1,Cidreira,-30.18,-50.21,68.25,82,14.54,98
2,Ribeira Grande,38.52,-28.70,61.16,65,13.22,46
3,Cayenne,4.93,-52.33,80.60,88,5.82,20
4,Hermanus,-34.42,19.23,48.99,89,10.22,63


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
locations = vacationDF[["Latitude", "Longitude"]]
humidity = vacationDF["Humidity (%)"].astype(float)
locations

,Latitude,Longitude
0,53.20,-8.57
1,-30.18,-50.21
2,38.52,-28.70
3,4.93,-52.33
4,-34.42,19.23
...,...,...
551,60.82,-115.80
552,21.28,82.82
553,48.60,-93.40
554,46.75,5.75


In [5]:
coords = vacationDF[["Latitude", "Longitude"]]
humidity = vacationDF["Humidity (%)"]

figure_layout = {
    'width': '1200px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout, map_type = "SATELLITE")


heat = gmaps.heatmap_layer(coords, weights = humidity, dissipating=False, max_intensity=500)
fig.add_layer(heat)

fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
 mask = (vacationDF["Temperature (F)"] < 79) & (vacationDF["Temperature (F)"] > 76) & (vacationDF["Wind Speed (mph)"] < 7)
vacaWeaDF = vacationDF.loc[mask].reset_index(drop=True)
vacaWeaDF.head()

,City,Latitude,Longitude,Temperature (F),Humidity (%),Wind Speed (mph),Cloudiness (%)
0,Terra Santa,-2.10,-56.49,77.77,87,2.28,88
1,Muscat,23.61,58.59,78.80,78,5.82,20
2,Nísia Floresta,-6.09,-35.21,76.14,88,5.82,40
3,Loandjili,-4.76,11.86,76.71,89,6.60,100
4,Carora,10.18,-70.08,78.31,81,1.79,95


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
vacaWeaDF.City.values

array(['Terra Santa', 'Muscat', 'Nísia Floresta', 'Loandjili', 'Carora',
       'Inírida', 'Santa Cruz de la Sierra', 'Kayes', 'Oranjestad',
       'Monrovia', 'Nyaunglebin', 'Cocorit', 'Takoradze', 'Esim',
       'Mocuba', 'Zabid', 'Iquitos', 'Lamu', 'Pitimbu', 'Mae Ramat',
       'Cabedelo', 'Yenagoa', 'Kathu', 'Luanda', 'Baillif'], dtype=object)

In [8]:
names = []
addresses = []

url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "rankby": "distance",
    "type": "hotel",
    "key": g_key,
}

for index, row in vacaWeaDF.iterrows():
    
    location = f"{row.Latitude}, {row.Longitude}"

    params['location'] = location

    print(f"Results for Index {index}: {row.City}.")
    response = requests.get(url, params=params).json()
    
    results = response['results']
    
    try:
        print(f"Closest Hotel is {results[0]['name']}.")
        
              
        name = results[0]['name']
        address = results[0]['vicinity']

        names.append(name)
        addresses.append(address)              
        
    except (KeyError, IndexError):
        print("Missing field")
        names.append(np.nan)
        addresses.append(np.nan)
        
    print("------------")

vacaWeaDF["Hotel Name"] = names
vacaWeaDF["Address"] = addresses

Results for Index 0: Terra Santa.
Closest Hotel is PIZZARIA DO ARTHUR DELIVERY.
------------
Results for Index 1: Muscat.
Closest Hotel is مقهى الزدجال.
------------
Results for Index 2: Nísia Floresta.
Closest Hotel is Salão do Reino das Testemunhas de Jeová.
------------
Results for Index 3: Loandjili.
Closest Hotel is COMMUNAUTE DES EGLISES LUMIERE DU MONDE (CELM).
------------
Results for Index 4: Carora.
Closest Hotel is Taller Electroauto Freddy.
------------
Results for Index 5: Inírida.
Closest Hotel is Corporación CDA.
------------
Results for Index 6: Santa Cruz de la Sierra.
Closest Hotel is Agencia El TRompillo.
------------
Results for Index 7: Kayes.
Missing field
------------
Results for Index 8: Oranjestad.
Closest Hotel is Ddepartemento Social Psychiatrico SPD.
------------
Results for Index 9: Monrovia.
Closest Hotel is Anglers Bar and Restaurant.
------------
Results for Index 10: Nyaunglebin.
Closest Hotel is Koe Na Win Zay Ti.
------------
Results for Index 11: Coc

In [9]:
hotel_df = vacaWeaDF
hotel_df.head()

,City,Latitude,Longitude,Temperature (F),Humidity (%),Wind Speed (mph),Cloudiness (%),Hotel Name,Address
0,Terra Santa,-2.10,-56.49,77.77,87,2.28,88,PIZZARIA DO ARTHUR DELIVERY,"Rua Barão do Rio Branco, 417-285 - Santa Clara..."
1,Muscat,23.61,58.59,78.80,78,5.82,20,مقهى الزدجال,"Unnamed Road, مسقط،"
2,Nísia Floresta,-6.09,-35.21,76.14,88,5.82,40,Salão do Reino das Testemunhas de Jeová,"Travessa Padre Basílio, 60-104 - Marco VII, Ní..."
3,Loandjili,-4.76,11.86,76.71,89,6.60,100,COMMUNAUTE DES EGLISES LUMIERE DU MONDE (CELM),99324
4,Carora,10.18,-70.08,78.31,81,1.79,95,Taller Electroauto Freddy,"Calle Lidice, Carora"


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

KeyError: 'Country'

In [11]:
infoList = []

for indx, row in hotel_df.iterrows():
    info = f"City: {row.City}<br>-----------<br>Hotel: {row['Hotel Name']}<br>-----------<br>Address: {row.Address}"
    infoList.append(info)

# Add marker layer ontop of heat map
coords = vacationDF[["Latitude", "Longitude"]]
coords_ideal = vacaWeaDF[["Latitude", "Longitude"]]
humidity = vacationDF["Humidity (%)"]

figure_layout = {
    'width': '1200px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout, map_type = "SATELLITE")

markers = gmaps.marker_layer(coords_ideal, info_box_content=infoList)
fig.add_layer(markers)

heat = gmaps.heatmap_layer(coords, weights = humidity, dissipating=False, max_intensity=500)
fig.add_layer(heat)


# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…